DATA SCHEMA

In [3]:
import psycopg2
from config_secrets import DATABASE_CREDENTIALS

# Function to create the 'Stories' table with the specified schema
def create_stories_table():
    try:
        conn = psycopg2.connect(**DATABASE_CREDENTIALS)

        # Open a cursor to execute SQL commands
        cursor = conn.cursor()

        # SQL command to create the 'Stories' table
        create_table_query = '''
        CREATE TABLE mythology (
            Culture_name TEXT,
            Title TEXT,
            Text TEXT
        );
        '''

        # Execute the SQL command to create the table
        cursor.execute(create_table_query)
        conn.commit()

        print("Table 'Stories' created successfully!")

    except (Exception, psycopg2.Error) as error:
        print("Error while connecting to PostgreSQL or executing the SQL command:", error)

    finally:
        if cursor:
            cursor.close()
        if conn:
            conn.close()

# Function to insert data into the 'Stories' table
def insert_data_into_stories_table(culture_name, title, text):
    try:
        conn = psycopg2.connect(**DATABASE_CREDENTIALS)

        # Open a cursor to execute SQL commands
        cursor = conn.cursor()

        # SQL command to insert data into the 'Stories' table
        insert_query = '''
        INSERT INTO mythology (Culture_name, Title, Text)
        VALUES (%s, %s, %s);
        '''

        # Execute the SQL command to insert data
        cursor.execute(insert_query, (culture_name, title, text))
        conn.commit()

        print(f"mythology '{title}' inserted successfully!")

    except (Exception, psycopg2.Error) as error:
        print("Error while connecting to PostgreSQL or inserting data:", error)

    finally:
        if cursor:
            cursor.close()
        if conn:
            conn.close()

if __name__ == "__main__":
    create_stories_table()

    # Example of inserting a story into the 'Stories' table
    example_title = "The Tale of the Mysterious Land"
    example_text = "Once upon a time, in a realm far away..."
    insert_data_into_stories_table("Culture1", example_title, example_text)


Table 'Stories' created successfully!
Story 'The Tale of the Mysterious Land' inserted successfully!


INSERTING DATA INTO DATABASE

In [4]:
import os
import psycopg2
import time
from config_secrets import DATABASE_CREDENTIALS

database_info = DATABASE_CREDENTIALS

local_directory = '/Users/francinasimone/Desktop/DreamApp/Starlight/First_People/'

def log_error(message):
    with open("error_log.txt", "a") as log_file:
        log_file.write(f"{time.strftime('%Y-%m-%d %H:%M:%S')} - {message}\n")

def validate_directory(path):
    """Checks if a given path is a directory."""
    return os.path.isdir(path)


def validate_file(file_path):
    if not os.path.isfile(file_path):
        log_error(f"File not found: {file_path}")
        return False
    return True

class DatabaseConnection:
    def __init__(self, db_info):
        self.db_info = db_info
        self.conn = None
        self.cursor = None

    def __enter__(self):
        self.conn = psycopg2.connect(**self.db_info)
        self.cursor = self.conn.cursor()
        return self.cursor

    def __exit__(self, exc_type, exc_val, exc_tb):
        self.conn.commit()
        self.cursor.close()
        self.conn.close()

def insert_data(culture_name, title, text):
    query = '''
    INSERT INTO mythology (Culture_name, Title, Text)
    VALUES (%s, %s, %s);
    '''
    with DatabaseConnection(database_info) as cursor:
        cursor.execute(query, (culture_name, title, text))

def store_stories_in_database(folder_names, directory):
    if not validate_directory(directory):
        return

    for culture_name in folder_names:
        culture_folder_path = os.path.join(directory, culture_name)

        # This is the primary check we're adding
        if not validate_directory(culture_folder_path):
            continue

        for story_file in os.listdir(culture_folder_path):
            story_path = os.path.join(culture_folder_path, story_file)

            # Here, we're also ensuring we only read valid files
            if os.path.isfile(story_path):
                with open(story_path, 'r') as f:
                    story_content = f.read()
                insert_data(culture_name, story_file, story_content)

folder_names = os.listdir(local_directory)  # We assume that all items are folders for simplicity.
store_stories_in_database(folder_names, local_directory)
